In [1]:
!pip install pymongo
!pip install tensorflow==2.16.1
!pip install keras==3.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found

In [2]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
from tqdm import tqdm

In [17]:
import json
with open('my_dict.json', 'r') as file:
    data = json.load(file)

In [4]:
client = MongoClient('mongodb://49.13.173.177:27020/')
embs = client.sponsoredbye.embeddings

In [ ]:
embeddings = []
labels = []

for n in tqdm(data['9']):
  temp_embs = []
  cursor = embs.find({'videoID':n})
  if cursor[0]['label'] == False:
    continue
  l = cursor[0]['label']
  e =  cursor[0]['embeddings']
  for m in e:
    temp_embs.append(m['embedding'])
  embeddings.append(temp_embs)
  labels.append(l)


 79%|███████▊  | 1558/1984 [15:08<03:41,  1.92it/s]

In [8]:
len(embeddings),len(labels)

(2943, 2943)

In [9]:
from keras.models import load_model
model = load_model('model_4.keras')

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Masking,TimeDistributed, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len = 1682
vector_dim = 768

X_padded = pad_sequences(embeddings,maxlen=max_len,dtype='float32', padding='post')
y_padded = pad_sequences(labels, maxlen=max_len, dtype='float32', padding='post')
y_padded = np.expand_dims(y_padded, -1)

In [ ]:
#model = Sequential()
#model.add(Masking(mask_value=0., input_shape=(max_len, vector_dim)))
#model.add(Bidirectional(LSTM(50, return_sequences=True)))
#model.add(Bidirectional(LSTM(50, return_sequences=True)))
#model.add(Dropout(0.2))
#model.add(TimeDistributed(Dense(1, activation='sigmoid')))
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_padded, y_padded, epochs=10, batch_size=4)

Epoch 1/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9861 - loss: 0.2895
Epoch 2/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9876 - loss: 0.2506
Epoch 3/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 185ms/step - accuracy: 0.9885 - loss: 0.2549
Epoch 4/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9897 - loss: 0.2314
Epoch 5/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9897 - loss: 0.2074
Epoch 6/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9903 - loss: 0.2062
Epoch 7/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 93s 187ms/step - accuracy: 0.9927 - loss: 0.1609
Epoch 8/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9922 - loss: 0.1700
Epoch 9/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 92s 186ms/step - accuracy: 0.9944 - loss: 0.1351
Epoch 10/10
371/496 ━━━━━━━━━━━━━━━━━━━━ 23s 185ms/step - accuracy: 0.9945 - loss: 0.1386

In [ ]:
model.save('model_5.keras')